# Selection of Calibration Windows for Day-Ahead Electricity Price Forecasting
## by Grzegorz Marcjasz, Tomasz Serafin and Rafał Weron 

In [3]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import median_abs_deviation

### 2. Methodology

#### 2.1. Preliminaries

In [5]:
def normalize_prices(prices):
    a = np.median(prices)
    b = median_abs_deviation(prices)
    p = 1/b * (prices - a)
    return p, a, b

def apply_vst(p): # apply the area hyperbolic sine transformation to the normalized prices
    X = np.arcsinh(p)
    return X

def inverse_transform(forecast, a, b): # apply the inverse of VST to the forecast to obtain the price prediction
    price_predictions = b * np.sinh(forecast) + a
    return price_predictions

In [ ]:
# Assuming array of prices
prices = np.array([...])

# Normalize prices
p, a, b = normalize_prices(prices)

# Apply VST
X = apply_vst(p)

# Steps for model training and forecasting go here

# Assuming forecasts is your array of forecasts
forecasts = np.array([...])

# Apply inverse transformation
price_predictions = inverse_transform(forecasts, a, b)

#### 2.2. Expert Models

ARX1

In [6]:
import statsmodels.api as sm

def prepare_data(data):
    # Prepare the data for the ARX1 model
    # This includes creating the lagged variables and the dummy variables for the weekdays
    data['X_lag1'] = data['X'].shift(1)
    data['X_lag2'] = data['X'].shift(2)
    data['X_lag7'] = data['X'].shift(7)
    data['X_min'] = data['X'].rolling(window=24).min().shift(24)
    data['load_forecast'] = data['load'].shift(24)
    data = pd.get_dummies(data, columns=['weekday'])
    data = data.dropna()  # drop missing values
    return data

def fit_arx1(data):
    # Fit the ARX1 model
    # The dependent variable is 'X'
    # The independent variables are the lagged variables, the load forecast, and the dummy variables
    exog_vars = ['X_lag1', 'X_lag2', 'X_lag7', 'X_min', 'load_forecast', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6']
    exog = sm.add_constant(data[exog_vars])
    endog = data['X']
    model = sm.OLS(endog, exog)
    results = model.fit()
    return results